In [1]:
import os 
import sys
import pandas as pd
import numpy as np

from bson import ObjectId

sys.path.append(os.path.abspath(os.path.join('..', '..')))

from services.pandas.pandas_service import PandasService
from services.database.database_service import Environment
from models.profile.profile_model import ProfileCollection

In [2]:
env = Environment().get_instance()
pd_services = PandasService()
db_users = env.get_db().get('users')
db_daily = env.get_db().get('daily')

2024-05-28 19:47:29,429 INFO root : Environment (__init__): making connection
2024-05-28 19:47:29,747 INFO root : Environment (__init__): making connection
2024-05-28 19:47:29,751 INFO root : Environment (get_db): access database
2024-05-28 19:47:29,752 INFO root : Environment (get_db): access database


In [3]:
user_collection = db_users.get_collection('UserCollection')
follow_collection = db_users.get_collection('FollowsCollection')
wh_collection = db_daily.get_collection('WhatHappenCollection')
auth_collection = db_users.get_collection('AuthCollection')

In [8]:
# for user in user_collection.find():
#     for happen in wh_collection.find({'user_id': user['_id']}):
#         happen['nickname'] = user['name']
#         wh_collection.replace_one({'_id': happen['_id']}, happen)

In [47]:
# for doc in user_collection.find():
#     doc['auth_id'] = ObjectId(doc['auth_id'])
#     user_collection.replace_one({'_id':doc['_id']}, doc)

In [43]:
list(follow_collection.find())

[]

In [23]:
list(user_collection.find({'_id': ObjectId('64f74f6e54d0ba6c4b532786')}))

[{'_id': ObjectId('64f74f6e54d0ba6c4b532786'),
  'email': 'kakashi.kisura7@gmail.com',
  'created_at': datetime.datetime(2023, 9, 5, 12, 55, 25, 8000),
  'updated_at': datetime.datetime(2024, 5, 10, 23, 16, 51, 337000),
  'auth_id': '663ed51204ba683d9499b096',
  'name_id': 'kakashi@daily',
  'name': 'kakashi',
  'profile_public': True,
  'genre': 'M',
  'phone': 5511941616008}]

In [40]:
list(follow_collection.find({'user_id': ObjectId('64f74f6e54d0ba6c4b532786')}))

[]

In [39]:
list(wh_collection.aggregate([
    {'$match': {'user_id': ObjectId('663ed74204ba683d9499b0a6')}},
    {'$lookup': {
        'as': 'feelings',
        'from': 'SentimentCollection',
        'localField': '_id',
        'foreignField': 'what_happen_id'
    }},
]))

[]

In [63]:
list(user_collection.find())

[{'_id': ObjectId('64f74f6e54d0ba6c4b532786'),
  'email': 'kakashi.kisura7@gmail.com',
  'created_at': datetime.datetime(2023, 9, 5, 12, 55, 25, 8000),
  'updated_at': datetime.datetime(2024, 5, 10, 23, 16, 51, 337000),
  'auth_id': '663ed51204ba683d9499b096',
  'name_id': 'kakashi@daily',
  'name': 'kakashi',
  'profile_public': True,
  'genre': 'M',
  'phone': 5511941616008},
 {'_id': ObjectId('65ae90883470eb010f5f1104'),
  'email': 'roberto.rosa7@gmail.com',
  'created_at': datetime.datetime(2024, 1, 22, 12, 58, 0, 736000),
  'updated_at': datetime.datetime(2024, 5, 7, 19, 10, 1, 134000),
  'auth_id': '663aa6b8dca08f4513232d7c',
  'name_id': 'beto@daily',
  'name': 'Beto',
  'profile_public': False,
  'genre': 'M',
  'phone': 5511941616008},
 {'_id': ObjectId('661f2d5e30ee4c696a1d15fd'),
  'email': 'root@gmail.com',
  'created_at': datetime.datetime(2024, 4, 17, 2, 1, 2, 908000),
  'updated_at': datetime.datetime(2024, 5, 4, 18, 44, 42, 291000),
  'auth_id': '6636ac49dca08f4513232d7

In [58]:
db_daily = env.get_db().get('daily')
sent_collection = db_daily.get_collection('SentimentCollection')

2024-05-16 00:12:40,983 INFO root : Environment (get_db): access database


In [ ]:
list(user_collection.find({ '$or': [ {'name_id': 'root@daily'}, {'email': 'root@gmail.com'} ], '$and': [{'profile_public': True}] } ))

In [ ]:

        # one = self.collection.find_one(ObjectId('65ae90883470eb010f5f1104'))
        # del one['roles']
        # one['updated_at'] = datetime.now()
        # one['auth_id'] = '661f2d5e30ee4c696a1d15fc'
        # one['name_id'] = 'beto@daily'
        # one['name'] = 'Beto'
        # one['profile_public'] = True
        # one['genre'] = 'M'
        # one['phone'] = '5511941616008'
        # print(one)
        # self.collection.replace_one({'_id': ObjectId('65ae90883470eb010f5f1104')}, one)


In [ ]:
show_columns = ['_id', 'email', 'name_id', 'profile_public', 'genre', 'phone', 'created_at', 'updated_at']
# dateparse = lambda x: datetime.strptime(x, '%Y/%m/%d %H:%M:%f.0000')
df_users = pd.DataFrame.from_records(user_collection.find(), columns=show_columns)
df_users

In [ ]:
df_users.info()

In [59]:
sents_columns = ['created_at', 'text', 'score_positive', 'score_negative', 'sentiment', '_id', 'what_happen_id', 'user_id', ]
df_sents = pd.DataFrame.from_records(sent_collection.find(),  columns=sents_columns)
df_sents.head()

,created_at,text,score_positive,score_negative,sentiment,_id,what_happen_id,user_id
0,2024-03-23 00:56:03.696,retomando aplicativo de diário,0.504341,0.495659,positive,65fe52d3aa58ed17295fa3d7,65fe52d3aa58ed17295fa3d6,65ae90883470eb010f5f1104
1,2024-03-23 08:37:46.714,Café da manhã no point dos amigos - rotina diá...,0.674941,0.325059,positive,65febf0aaa58ed17295fa3fb,65febf0aaa58ed17295fa3fa,65ae90883470eb010f5f1104
2,2024-03-23 11:55:57.605,tedsy,0.500758,0.499242,positive,65feed7daa58ed17295fa4bf,65feed7daa58ed17295fa4be,65ae90883470eb010f5f1104
3,2024-03-23 13:54:17.095,feito teste com tablet - sucesso,0.597084,0.402916,positive,65ff0939aa58ed17295fa509,65ff0938aa58ed17295fa508,65ae90883470eb010f5f1104
4,2024-03-23 16:32:58.342,mais um Dev fazendo coisa errada no projeto,0.214928,0.785072,negative,65ff2e6a2306ca0842f7cfff,65ff2e692306ca0842f7cffe,65ae90883470eb010f5f1104


In [ ]:
# df_users['created_at'] = pd.to_datetime(df_users['created_at'], format = '%Y-%m-%d %H:%M:%S.%f.000')
# df_users['updated_at'] = pd.to_datetime(df_users['updated_at'], format = '%Y-%m-%d %H:%M:%S.%f.000')
df_users

In [ ]:
df_sents.info()

In [ ]:
# df_sents.query("user_id == '65ae90883470eb010f5f1104'")
df_sents.query("sentiment == 'positive'").shape[0]

In [60]:
df_sents = pd.DataFrame.from_records(sent_collection.find({'user_id': '663ed5f504ba683d9499b09d'}),  columns=sents_columns)
df_sents.head()

,created_at,text,score_positive,score_negative,sentiment,_id,what_happen_id,user_id


https://konfuzio.com/en/nan-python-circumvent-with-nan-values-in-python/

In [ ]:
df_sents.value_counts().count()

In [ ]:
df_sents['sentiment'].value_counts().mean()

In [ ]:
profile = {}

name = 'sentiment'

profile['total_registers'] = df_sents.value_counts().count()
profile['total_positive'] = df_sents.query(f"{name} == 'positive'").shape[0]
profile['total_negative'] = df_sents.query(f"{name} == 'negative'").shape[0]
profile['total_score_negative'] = df_sents['score_negative'].sum()
profile['total_score_positive'] = df_sents['score_positive'].sum()
profile['mean_feeling'] = df_sents[name].value_counts().mean()
profile['std_feeling'] = 0 if np.isnan(df_sents[name].value_counts().std()) else df_sents[name].value_counts().std() 
profile['chart'] = df_sents[name].value_counts().map(lambda x: (x / df_sents.value_counts().count()) * 100).to_dict()
profile

In [ ]:
df_sents['score_negative'].sum()

In [ ]:
profile = vars((ProfileCollection()
 .set_chart(df_sents[name].value_counts().map(lambda x: (x / df_sents.value_counts().count()) * 100).to_dict())
 .set_std_feeling(0 if np.isnan(df_sents[name].value_counts().std()) else df_sents[name].value_counts().std())
 .set_mean_feeling(df_sents[name].value_counts().mean())
 .set_total_score_negetive(df_sents['score_negative'].sum())
 .set_total_score_positive(df_sents['score_positive'].sum())
 .set_total_negative(df_sents.query(f"{name} == 'negative'").value_counts().count())
 .set_total_positive(df_sents.query(f"{name} == 'positive'").value_counts().count())
 .set_total_registers(df_sents.value_counts().count())))

profile

In [ ]:
list(follow_collection.find())

In [98]:
list(follow_collection.aggregate([
    {"$group": {"_id": "$name", "data": { "$first": "$$ROOT"},
                "following": { "$sum": {"$cond": [{"$eq": ["$$ROOT.user_id",'661f2d5e30ee4c696a1d15fd']},1,0]}},
                "followers": {"$sum": {"$cond": [{"$eq": ["$$ROOT.following_id",'661f2d5e30ee4c696a1d15fd']},1,0]}}}},
    {"$project": {"total_following": "$following","total_followers": "$followers"}},
    {'$unset': ['_id']}]))

[{'total_following': 2, 'total_followers': 3}]

In [113]:
list(follow_collection.aggregate([
    {"$group": {"_id": "$name", "data": { "$first": "$$ROOT"},
                "reject": {"$sum":{"$cond": [ {'$and':[{'$eq': ['$$ROOT.user_id','661f2d5e30ee4c696a1d15fd']}, {'$eq': ['$$ROOT.status', 'REJECT']}]},1,0 ]}},
                "request": {"$sum":{"$cond": [ {'$and':[{'$eq': ['$$ROOT.user_id','661f2d5e30ee4c696a1d15fd']}, {'$eq': ['$$ROOT.status', 'REQUESTED']}]},1,0 ]}},
                "cancel": {"$sum":{"$cond": [ {'$and':[{'$eq': ['$$ROOT.user_id','661f2d5e30ee4c696a1d15fd']}, {'$eq': ['$$ROOT.status', 'CANCEL_REQUEST']}]},1,0 ]}},
                "following": {"$sum":{"$cond": [ {'$and':[{'$eq': ['$$ROOT.user_id','661f2d5e30ee4c696a1d15fd']}, {'$eq': ['$$ROOT.status', 'FOLLOWING']}]},1,0 ]}},
                "followers": {"$sum":{"$cond": [ {'$and':[{'$eq': ['$$ROOT.following_id','661f2d5e30ee4c696a1d15fd']}, {'$eq': ['$$ROOT.status', 'FOLLOWING']}]},1,0 ]}}}},
    {"$project": {"following": "$following","followers": "$followers", 'reject': '$reject', 'request': '$request', 'cancel': '$cancel', 'follow':'$follow'}},
    {'$unset': ['_id']}]))

[{'following': 2, 'followers': 3, 'reject': 0, 'request': 0, 'cancel': 0}]

In [56]:
list(follow_collection.find({"user_id": "64f74f6e54d0ba6c4b532786", 'following_id': '663ed5f504ba683d9499b09d'}))

[{'_id': ObjectId('664411eaff47ba18dd110275'),
  'user_id': '64f74f6e54d0ba6c4b532786',
  'following_id': '663ed5f504ba683d9499b09d',
  'created_at': datetime.datetime(2024, 5, 15, 1, 37, 46, 472000),
  '_class': 'innovatenet.com.br.document.FollowsDocument',
  'status': 'FOLLOW'}]

In [33]:
list(user_collection.aggregate([
    { '$lookup': { 'from': "FollowsCollection", 'localField': "_id", 'foreignField': "user_id", 'as': "custom_auth"} },
    # { '$unwind': "$custom_auth" }
]))

[{'_id': ObjectId('64f74f6e54d0ba6c4b532786'),
  'email': 'kakashi.kisura7@gmail.com',
  'created_at': datetime.datetime(2023, 9, 5, 12, 55, 25, 8000),
  'updated_at': datetime.datetime(2024, 5, 10, 23, 16, 51, 337000),
  'auth_id': '663ed51204ba683d9499b096',
  'name_id': 'kakashi@daily',
  'name': 'kakashi',
  'profile_public': True,
  'genre': 'M',
  'phone': 5511941616008,
  'custom_auth': []},
 {'_id': ObjectId('65ae90883470eb010f5f1104'),
  'email': 'roberto.rosa7@gmail.com',
  'created_at': datetime.datetime(2024, 1, 22, 12, 58, 0, 736000),
  'updated_at': datetime.datetime(2024, 5, 7, 19, 10, 1, 134000),
  'auth_id': '663aa6b8dca08f4513232d7c',
  'name_id': 'beto@daily',
  'name': 'Beto',
  'profile_public': False,
  'genre': 'M',
  'phone': 5511941616008,
  'custom_auth': []},
 {'_id': ObjectId('661f2d5e30ee4c696a1d15fd'),
  'email': 'root@gmail.com',
  'created_at': datetime.datetime(2024, 4, 17, 2, 1, 2, 908000),
  'updated_at': datetime.datetime(2024, 5, 4, 18, 44, 42, 2910